# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116056e+02     1.577682e+00
 * time: 1.1378211975097656
     1     1.067926e+01     9.056192e-01
 * time: 3.4663472175598145
     2    -1.131946e+01     9.913793e-01
 * time: 4.098193168640137
     3    -3.422928e+01     8.071643e-01
 * time: 5.055798053741455
     4    -4.742222e+01     6.709342e-01
 * time: 5.96961522102356
     5    -5.682644e+01     2.526858e-01
 * time: 6.927574157714844
     6    -5.965997e+01     2.395867e-01
 * time: 7.567251205444336
     7    -6.084517e+01     9.476527e-02
 * time: 8.205366134643555
     8    -6.139029e+01     4.560247e-02
 * time: 8.831188201904297
     9    -6.170943e+01     3.827996e-02
 * time: 9.455083131790161
    10    -6.192500e+01     2.700223e-02
 * time: 10.125914096832275
    11    -6.204075e+01     2.158195e-02
 * time: 10.784012079238892
    12    -6.211181e+01     1.774832e-02
 * time: 11.405257225036621
    13    -6.215154e+01     1.414801e-02
 * time: 12.036683082580566
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666461969
